Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

In [3]:
sales.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    print (block_num)
    print (cur_shops)
    print (cur_items)
    al =  (list(product(cur_shops, cur_items,[block_num])))
    print(len(cur_shops))
    print(len(cur_items))
    print (len(cur_items)*len(cur_shops))
    print(len(al))
    print(np.asarray(al))
    df_al = pd.DataFrame(np.vstack(al), columns = index_cols,dtype=np.int32)
    print(df_al.head())
    break


0
[59 25 24 23 19 22 18 21 28 27 29 26  4  6  2  3  7  0  1 16 15  8 10 14 13
 12 53 31 30 32 35 56 54 47 50 42 43 52 51 41 38 44 37 46 45]
[22154  2552  2554 ...,   628   631   621]
45
8115
365175
365175
[[   59 22154     0]
 [   59  2552     0]
 [   59  2554     0]
 ..., 
 [   45   628     0]
 [   45   631     0]
 [   45   621     0]]
   shop_id  item_id  date_block_num
0       59    22154               0
1       59     2552               0
2       59     2554               0
3       59     2555               0
4       59     2564               0


In [5]:
sales.loc[(sales["date_block_num"]==1) & (sales["item_id"]==30) & (sales["shop_id"]==0)]

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
173344  21.02.2013               1        0       30       265.0           2.0
173345  20.02.2013               1        0       30       265.0           2.0
173346  18.02.2013               1        0       30       265.0           4.0
173347  17.02.2013               1        0       30       265.0           4.0
173348  16.02.2013               1        0       30       265.0           9.0
173349  15.02.2013               1        0       30       265.0           2.0
173361  22.02.2013               1        0       30       265.0           2.0
173402  23.02.2013               1        0       30       265.0           3.0
173441  26.02.2013               1        0       30       265.0           3.0

In [6]:
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'sum'}})
gb.head()

shop_id item_id date_block_num item_cnt_day
                                          sum
0       0      30              1         31.0
1       0      31              1         11.0
2       0      32              0          6.0
3       0      32              1         10.0
4       0      33              0          3.0

In [7]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
all_data.head()

shop_id  item_id  date_block_num  target
139255        0       19               0     0.0
141495        0       27               0     0.0
144968        0       28               0     0.0
142661        0       29               0     0.0
138947        0       32               0     6.0

In [9]:
all_data.describe()

shop_id       item_id  date_block_num        target
count  1.091385e+07  1.091385e+07    1.091385e+07  1.091385e+07
mean   3.118720e+01  1.130926e+04    1.497334e+01  3.342731e-01
std    1.734959e+01  6.209978e+03    9.495618e+00  3.417243e+00
min    0.000000e+00  0.000000e+00    0.000000e+00 -2.200000e+01
25%    1.600000e+01  5.976000e+03    7.000000e+00  0.000000e+00
50%    3.000000e+01  1.139100e+04    1.400000e+01  0.000000e+00
75%    4.600000e+01  1.660500e+04    2.300000e+01  0.000000e+00
max    5.900000e+01  2.216900e+04    3.300000e+01  2.253000e+03

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [10]:
np.round(all_data.target.mean(),decimals=4)

0.33429999999999999

In [11]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(all_data.target.mean(), inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


#### Method 2

In [12]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(all_data.target.mean(), inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [13]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [14]:
all_data.shape

(10913850, 5)

In [15]:
import sklearn
from sklearn.model_selection import KFold

kf = KFold(5, shuffle=False)

col = "item_target_kfold_mean"
all_data[col] = np.nan                                     

for trn_ind, val_ind in kf.split(all_data):
    trn, val = all_data.iloc[trn_ind], all_data.iloc[val_ind]
    val[col]=  val['item_id'].map(trn.groupby("item_id").target.mean())
    all_data.iloc[val_ind, :] = val

all_data["item_target_kfold_mean"].fillna(0.3343, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
# YOUR CODE GOES HERE

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, all_data["item_target_kfold_mean"].values)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.41645907128
Current answer for task KFold_scheme is: 0.41645907128


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [17]:
# YOUR CODE GOES HERE
col = "item_target_LOO_mean"
all_data[col] = np.nan  
grp = all_data.groupby("item_id").target
all_data_group_sum = all_data["item_id"].map(grp.sum())
all_data_group_count = all_data["item_id"].map(grp.count())
all_data[col] = (all_data_group_sum - all_data["target"])/ (all_data_group_count-1)

corr = np.corrcoef(all_data['target'].values, all_data[col])[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [30]:
# YOUR CODE GOES HERE

col = "item_target_smooth_mean"
all_data[col] = np.nan  
global_mean = all_data.target.mean()

alpha=100
group_rows = all_data.groupby("item_id").target.count()
group_mean = all_data.groupby("item_id").target.mean()

smooth_mean = (group_rows* group_mean + alpha*global_mean)/(group_rows+alpha)
all_data[col] = all_data['item_id'].map(smooth_mean)
all_data[col].fillna(0.3343, inplace = True)

corr = np.corrcoef(all_data['target'].values, all_data[col].values)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.481819889227
Current answer for task Smoothing_scheme is: 0.481819889227


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [19]:
# YOUR CODE GOES HERE
col = "item_target_expanding_mean"
all_data[col] = np.nan 

cumsum = all_data.groupby("item_id").target.cumsum() - all_data.target
cumcount = all_data.groupby("item_id").target.cumcount()
all_data[col] = cumsum/cumcount
all_data[col].fillna(0.3343, inplace = True)
corr = np.corrcoef(all_data['target'].values, all_data[col])[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.502524521108
Current answer for task Expanding_mean_scheme is: 0.502524521108


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [31]:
STUDENT_EMAIL = "aakashsingh@gmail.com"
STUDENT_TOKEN = "HUgmgpUUWt7YEf5E"
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907128
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.481819889227
Task Expanding_mean_scheme: 0.502524521108


In [32]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
